In [ ]:
#this works perfectly!!

import cv2
import mediapipe as mp
import numpy as np
import serial
import time
import requests
# Initialize Mediapipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Initialize Serial Communication (Update COM Port)
arduino_port = 'COM9'  # Change to '/dev/ttyUSB0' for Linux/macOS
baud_rate = 9600
try:
    ser = serial.Serial(arduino_port, baud_rate, timeout=1)
    time.sleep(2)  # Allow time for connection to establish
    print(f"Connected to {arduino_port}")
except Exception as e:
    print(f"Error connecting to Arduino: {e}")
    ser = None

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

# Variables for rep tracking and form feedback
rep_count = 0
squat_state = "UP"
feedback = ""
feedback_good_bad = ""

# Load Video
cap = cv2.VideoCapture('training_data.mp4')

# Start Pose Detection
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # Stop if video ends

       # frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)

        # Convert frame to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Detect pose
        results = pose.process(image)

        # Convert back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            h, w, _ = image.shape  # Get frame dimensions

            # Get coordinates of relevant joints
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST].x * w,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST].y * h]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP].x * w,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP].y * h]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE].x * w,
                         landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y * h]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].x * w,
                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].y * h]

            # ✅ Calculate angles
            angle_hip_knee = calculate_angle(left_wrist, left_hip, left_knee)
            angle_knee_ankle = calculate_angle(left_hip, left_knee, left_ankle)
            angle_diff = angle_hip_knee - angle_knee_ankle

            # ✅ Compensation Detection Logic
            prev_feedback_good_bad = feedback_good_bad  # Store previous feedback state
            feedback = ""  # Store multiple feedback messages
            feedback_good_bad = ""  # Default to good form

            if angle_knee_ankle < 80:  # Deep squat position
                if angle_diff < -10:  
                    # ✅ Forward Lean Compensation
                    feedback = "Keep your chest up"
                    feedback_good_bad = "bad"

                elif left_knee[0] > left_ankle[0] + 30:  
                    # ✅ Knees Over Toes Compensation
                    feedback = "Push your hips back, your knees should not go beyond your toes"
                    feedback_good_bad = "bad"

            else:  
                # ✅ Good Form
                feedback = "Good Form"
                feedback_good_bad = "good"

            # ✅ Send Feedback to Arduino IMMEDIATELY when it changes to "bad"
            if feedback_good_bad == "bad" and prev_feedback_good_bad != "bad":
                if ser:
                    ser.write(b"bad\n")  # Send "bad" immediately
                    print("⚠️ Sent to Arduino: bad")



            # ✅ Rep Counting and LED Logic
            if angle_knee_ankle > 160:  # Standing (UP position)
                if squat_state == "DOWN":  # Completed full squat cycle (DOWN → UP)
                    rep_count += 1
                    print(f"✅ Rep Count: {rep_count}")

                    # ✅ Send feedback to Arduino **only after full rep**
                    if ser:
                        message = f"{feedback_good_bad}\n" if feedback_good_bad else "\n"  # Send empty signal when null
                        ser.write(message.encode())  
                        print(f"Sent to Arduino: {message.strip()}")

                # ✅ Reset feedback and form when returning to standing position
                feedback = ""  # Clear feedback message
                feedback_good_bad = ""  # Reset form status

                squat_state = "UP"

            elif angle_knee_ankle < 90:  # Deep squat (DOWN position)
                squat_state = "DOWN"


            # ✅ Draw pose landmarks
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # ✅ Display knee angle at knee position
            knee_x, knee_y = int(left_knee[0]), int(left_knee[1])
            cv2.putText(image, f"{int(angle_knee_ankle)}°", (knee_x, knee_y), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)

            # ✅ Display Feedback & Rep Counter
            cv2.rectangle(image, (0, 0), (400, 100), (245, 117, 16), -1)  # Background

            cv2.putText(image, f"Reps: {rep_count}", (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2, cv2.LINE_AA)

            cv2.putText(image, f"Feedback: {feedback}", (10, 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0) if feedback_good_bad == "good" else (0, 0, 255), 2, cv2.LINE_AA)

            cv2.putText(image, f"Form: {feedback_good_bad}", (10, 90), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0) if feedback_good_bad == "good" else (0, 0, 255), 2, cv2.LINE_AA)

        # Display the video
        cv2.imshow('Squat Counter & Form Feedback', image)

        # Exit on 'q' key
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Release resources
cap.release()
cv2.destroyAllWindows()
if ser:
    ser.close()


Connected to COM9
✅ Rep Count: 1
Sent to Arduino: good


In [4]:
import cv2
import mediapipe as mp
import numpy as np
import serial
import time
import requests

URL_Good_form = "https://api-v2.voicemonkey.io/flows?token=397bcb3a6c477c58539dbb6a425b835e_2873419b63829468723895a14116e211&flow=1003"
URL_knees_beyond_toes = "https://api-v2.voicemonkey.io/flows?token=397bcb3a6c477c58539dbb6a425b835e_2873419b63829468723895a14116e211&flow=1001"
URL_lean_forward = "https://api-v2.voicemonkey.io/flows?token=397bcb3a6c477c58539dbb6a425b835e_2873419b63829468723895a14116e211&flow=1002"

# Initialize Mediapipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Initialize Serial Communication (Update COM Port)
arduino_port = 'COM9'  # Change to '/dev/ttyUSB0' for Linux/macOS
baud_rate = 9600
try:
    ser = serial.Serial(arduino_port, baud_rate, timeout=1)
    time.sleep(2)  # Allow time for connection to establish
    print(f"Connected to {arduino_port}")
except Exception as e:
    print(f"Error connecting to Arduino: {e}")
    ser = None

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

# Variables for rep tracking and form feedback
rep_count = 0
squat_state = "UP"
feedback = ""
feedback_good_bad = ""
bad_forward_lean = False
bad_knees_over_toes = False

# Load Video
cap = cv2.VideoCapture("all.mp4")
#cap = cv2.VideoCapture(1)  # 0 is the default camera (built-in webcam)


# Start Pose Detection
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # Stop if video ends

        # Convert frame to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Detect pose
        results = pose.process(image)

        # Convert back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            h, w, _ = image.shape  # Get frame dimensions

            # Get coordinates of relevant joints
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST].x * w,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST].y * h]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP].x * w,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP].y * h]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE].x * w,
                         landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y * h]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].x * w,
                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].y * h]

            # ✅ Calculate angles
            angle_hip_knee = calculate_angle(left_wrist, left_hip, left_knee)
            angle_knee_ankle = calculate_angle(left_hip, left_knee, left_ankle)
            angle_diff = angle_hip_knee - angle_knee_ankle

            # ✅ Compensation Detection Logic
            prev_feedback_good_bad = feedback_good_bad  # Store previous feedback state
            feedback = ""  # Store multiple feedback messages
            feedback_good_bad = "good"  # Default to good form
            bad_forward_lean = False
            bad_knees_over_toes = False

            if angle_knee_ankle < 80:  # Deep squat position
                if angle_diff < -10:  
                    # ✅ Forward Lean Compensation
                    feedback = "Keep your chest up"
                    feedback_good_bad = "bad_forward_lean"
                    bad_forward_lean = True

                if left_knee[0] > left_ankle[0] + 30:  
                    # ✅ Knees Over Toes Compensation
                    feedback = "Push your hips back, your knees should not go beyond your toes"
                    feedback_good_bad = "bad_knees_over_toes"
                    bad_knees_over_toes = True

            # ✅ Send Feedback to Arduino IMMEDIATELY when bad form is detected
            if bad_forward_lean and feedback_good_bad != prev_feedback_good_bad:
                if ser:
                    ser.write(b"bad_forward_lean\n")  # Send "bad_forward_lean"
                    requests.get(URL_lean_forward)
                    print("⚠️ Sending to Alexa: Forward lean correction")
                    print("⚠️ Sent to Arduino: bad_forward_lean")

            if bad_knees_over_toes and feedback_good_bad != prev_feedback_good_bad:
                if ser:
                    ser.write(b"bad_knees_over_toes\n")  # Send "bad_knees_over_toes"
                    requests.get(URL_knees_beyond_toes)
                    print("⚠️ Sending to Alexa: Knees Beyond Toes Correction")
                    print("⚠️ Sent to Arduino: bad_knees_over_toes")

            # # ✅ Rep Counting and LED Logic
            # if angle_knee_ankle > 160:  # Standing (UP position)
            #     if squat_state == "DOWN":  # Completed full squat cycle (DOWN → UP)
            #         rep_count += 1
            #         print(f"✅ Rep Count: {rep_count}")

            #         # ✅ Send feedback to Arduino **only after full rep**
            #         if ser:
            #             message = f"{feedback_good_bad}\n" if feedback_good_bad else "\n"  # Send empty signal when null
            #             ser.write(message.encode())  
            #             requests.get(URL_Good_form)
            #             print("⚠️ Sending to Alexa: Good Form")
            #             print(f"Sent to Arduino: {message.strip()}")

            #     # ✅ Reset feedback and form when returning to standing position
            #     feedback = ""  # Clear feedback message
            #     feedback_good_bad = "good"  # Reset form status
            #     squat_state = "UP"

            # elif angle_knee_ankle < 90:  # Deep squat (DOWN position)
            #     squat_state = "DOWN"
            if angle_knee_ankle > 160:  # Standing (UP position)
                if squat_state == "DOWN":  # Completed full squat cycle (DOWN → UP)
                    rep_count += 1
                    print(f"✅ Rep Count: {rep_count}")

                    # ✅ Send only **one** Alexa request based on worst detected form during the squat
                    if ser:
                        if bad_knees_over_toes:
                            ser.write(b"bad_knees_over_toes\n")  # Send "bad_knees_over_toes"
                            requests.get(URL_knees_beyond_toes)
                            print("⚠️ Sending to Alexa: Knees Beyond Toes Correction")
                            print("⚠️ Sent to Arduino: bad_knees_over_toes")

                        elif bad_forward_lean:
                            ser.write(b"bad_forward_lean\n")  # Send "bad_forward_lean"
                            requests.get(URL_lean_forward)
                            print("⚠️ Sending to Alexa: Forward lean correction")
                            print("⚠️ Sent to Arduino: bad_forward_lean")

                        else:  # If no bad form was detected, send "Good Form"
                            ser.write(b"good\n")  
                            requests.get(URL_Good_form)
                            print("⚠️ Sending to Alexa: Good Form")
                            print("Sent to Arduino: good")

                # ✅ Reset feedback after completing the rep
                feedback = ""
                feedback_good_bad = "good"
                squat_state = "UP"

            elif angle_knee_ankle < 90:  # Deep squat (DOWN position)
                squat_state = "DOWN"

            # ✅ Draw pose landmarks
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # ✅ Display knee angle at knee position
            knee_x, knee_y = int(left_knee[0]), int(left_knee[1])
            cv2.putText(image, f"{int(angle_knee_ankle)}°", (knee_x, knee_y), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)

            # ✅ Display Feedback & Rep Counter
            cv2.rectangle(image, (0, 0), (400, 100), (245, 117, 16), -1)  # Background

            cv2.putText(image, f"Reps: {rep_count}", (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2, cv2.LINE_AA)

            cv2.putText(image, f"Feedback: {feedback}", (10, 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0) if feedback_good_bad == "good" else (0, 0, 255), 2, cv2.LINE_AA)

            cv2.putText(image, f"Form: {feedback_good_bad}", (10, 90), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0) if feedback_good_bad == "good" else (0, 0, 255), 2, cv2.LINE_AA)

        # Display the video
        cv2.imshow('Squat Counter & Form Feedback', image)

        # Exit on 'q' key
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Release resources
cap.release()
cv2.destroyAllWindows()
if ser:
    ser.close()


Connected to COM9
✅ Rep Count: 1
⚠️ Sending to Alexa: Good Form
Sent to Arduino: good
⚠️ Sending to Alexa: Knees Beyond Toes Correction
⚠️ Sent to Arduino: bad_knees_over_toes
✅ Rep Count: 2
⚠️ Sending to Alexa: Good Form
Sent to Arduino: good
⚠️ Sending to Alexa: Forward lean correction
⚠️ Sent to Arduino: bad_forward_lean
⚠️ Sending to Alexa: Knees Beyond Toes Correction
⚠️ Sent to Arduino: bad_knees_over_toes
✅ Rep Count: 3
⚠️ Sending to Alexa: Good Form
Sent to Arduino: good


In [ ]:
# ✅ Rep Counting and LED Logic
if angle_knee_ankle > 160:  # Standing (UP position)
    if squat_state == "DOWN":  # Completed full squat cycle (DOWN → UP)
        rep_count += 1
        print(f"✅ Rep Count: {rep_count}")

        # ✅ Send only **one** Alexa request based on worst detected form during the squat
        if ser:
            if bad_knees_over_toes:
                ser.write(b"bad_knees_over_toes\n")  # Send "bad_knees_over_toes"
                requests.get(URL_knees_beyond_toes)
                print("⚠️ Sending to Alexa: Knees Beyond Toes Correction")
                print("⚠️ Sent to Arduino: bad_knees_over_toes")

            elif bad_forward_lean:
                ser.write(b"bad_forward_lean\n")  # Send "bad_forward_lean"
                requests.get(URL_lean_forward)
                print("⚠️ Sending to Alexa: Forward lean correction")
                print("⚠️ Sent to Arduino: bad_forward_lean")

            else:  # If no bad form was detected, send "Good Form"
                ser.write(b"good\n")  
                requests.get(URL_Good_form)
                print("⚠️ Sending to Alexa: Good Form")
                print("Sent to Arduino: good")

    # ✅ Reset feedback after completing the rep
    feedback = ""
    feedback_good_bad = "good"
    squat_state = "UP"

elif angle_knee_ankle < 90:  # Deep squat (DOWN position)
    squat_state = "DOWN"
